# How function works in python ?
Reference: https://lerner.co.il/2020/04/29/function-dissection-lab-learn-how-python-functions-work-by-examining-their-innards/

function are objects.<br>
- can be assigned.<br>
hello2 = hello
- can be passed as arguments.<br>
hello( hello )
- can have attributes.<br>
dir( hello )


## How attributes used in the function

In [2]:
def hello(name):
    return f'Hello, {name}'

hello("harsha")

'Hello, harsha'

In [3]:
hello()

TypeError: hello() missing 1 required positional argument: 'name'

Observation: How function knows, it needs attributes ? and also attribute as *name* ?<br>
> `__code__` 

It has Python byte code and Hints to py interpreter about the function.

In [6]:
# this prints numbers of args required to run function
# while running 

hello.__code__.co_argcount

1

In [9]:
# __code__.co_varnames

hello.__code__.co_varnames # TUple and lists all of a function local variables 

('name',)

So, function knows, lists of variables required and if no arguments passed will throw `Missing variables`.

## Two parameters

In [11]:

def hello( first, last ):
    return f'Hello, {first} {last}'


print( hello.__code__.co_argcount ) # Number of fuction local variables

print( hello.__code__.co_varnames )

2
('first', 'last')


In [12]:

hello('harsha')


TypeError: hello() missing 1 required positional argument: 'last'

In [13]:
hello( last='vardhana' )

TypeError: hello() missing 1 required positional argument: 'first'

In [15]:
hello( 'a', 'b', 'c' ) # when greater variables 

TypeError: hello() takes 2 positional arguments but 3 were given

## Additional variables inside the function


In [16]:

def hello( first, last ):
    s = f'Hello, {first}{last}!'
    return s



In [19]:
print( hello.__code__.co_argcount  ) # Expected args for the function

print( hello.__code__.co_varnames ) # number of parameters inside the function

2
('first', 'last', 's')


## what is *args

In [25]:

def hello( first, last, *args ):
    return f'Hello {first} {last}, args = {args}'


print( hello( 'a','b', 'c', 'd' , 'e' ) ) # How *args knows the args 

print( hello.__code__.co_argcount ) # Why only 2 args ?

print( hello.__code__.co_varnames ) 




Hello a b, args = ('c', 'd', 'e')
2
('first', 'last', 'args')


###  there are 3 required args inside the function ? and tuple is not args ? How tuple and dict args managed ?<br>

This information is kept in `co_flags`, a **int**.<br>
This **int** is the bitwise "and" of several bit flags.<br>

-------------------------------------------------------------- <br>
| 2^5       | 2^4    | 2^3      | 2^2    |   2^1     | 2^0       | <br>
| Generator | Nested | **kwargs | *args  | New locals | Optimized| 


 co_optimized    0x01    # use fast locals<br>
 co_newlocals    0x02    # new dict for code block <br>
 co_varargs      0x04    # function has *args <br>
 co_varkeywords  0x08    # function has **kwargs <br>
 co_nested       0x10    # nested scopes <br>
 co_generator    0x20    # it’s a generator function<br>

In [27]:
hello.__code__.co_flags & 0x04  # Yes *ags # due to tuple

4

In [28]:
hello.__code__.co_flags & 0x08 # NO  **kwargs # due to dict

0

## **kwargs

In [32]:
def hello( **kwargs ):
    return f'hello, {kwargs}! '


print( hello.__code__.co_flags & 0x04  )
    
    
print(  hello.__code__.co_flags & 0x08 ) # due to dict

0
8


## Another method using `dis`

In [36]:
import dis 

dis.show_code(hello) # this shows by defaults tuple or dict

Name:              hello
Filename:          <ipython-input-32-fb9da898478d>
Argument count:    0
Kw-only arguments: 0
Number of locals:  1
Stack size:        3
Flags:             OPTIMIZED, NEWLOCALS, VARKEYWORDS, NOFREE
Constants:
   0: None
   1: 'hello, '
   2: '! '
Variable names:
   0: kwargs


## How to manage `Constants` inside the function  ? <br>

Literal values are stored in `__code__.co_consts` <br>


In [37]:
hello.__code__.co_consts 

(None, 'hello, ', '! ')

## Bytecodes

In [39]:
 hello.__code__.co_code  # cann't read. so,use  dis.dis
    

b'd\x01|\x00\x9b\x00d\x02\x9d\x03S\x00'

In [40]:
dis.dis(hello) 

  2           0 LOAD_CONST               1 ('hello, ')
              2 LOAD_FAST                0 (kwargs)
              4 FORMAT_VALUE             0
              6 LOAD_CONST               2 ('! ')
              8 BUILD_STRING             3
             10 RETURN_VALUE


## what about defaults ?

In [41]:

def hello( name='world' ):
    return f' hello, {name}' 

hello.__code__.co_argcount 

1

## ` __defaults__` <br>

• A function’s defaults are stored in `__defaults__`  <br>

`__defaults__` is always a tuple. <br>

No defaults? Then it’s an empty tuple.

In [42]:
hello.__defaults__ 

('world',)

## what happens when function calls ? <br>



• It compares the arguments with co_argcount. <br>
• Does the number match? <br>
• Pass arguments and call the function. <br>
• Not enough arguments? <br>
• Checks if __defaults__ can close the gap. <br>
• If so, use enough from __defaults__ to get to co_argcount. <br>
• Too many arguments? <br>
• Check co_flags to see if *args is defined. <br>
• If so, assign remaining arguments to *args. <br>
• Or whatever variable is named in co_varnames[co_argcount] <br>


In [46]:

def add_one(x):
     x.append(1) 
        

mylist = [10, 20, 30] 
add_one(mylist) 
print( mylist )

print()

add_one(mylist) 
print( mylist ) # why added extra '1'


[10, 20, 30, 1]

[10, 20, 30, 1, 1]


## Let's add a default


In [48]:
def add_one(x=[]):
    x.append(1)
    return x 

print(add_one()) 
print(add_one()) 
print(add_one()) 

[1]
[1, 1]
[1, 1, 1]


## Why this Problem ?
this is due to `__defaults__` 

Conclusion: <br>
Never use mutable defaults!.

> Don’t ignore this warning!

``` python
$ pylint add_one.py
************* Module add_one
add_one.py:4:0: W0102: Dangerous default value [] as
argument (dangerous-default-value)

```

## Keyword-only arguments

In [49]:

def hello(*args, sep=' '):
    return f'Hello, {sep.join(args)}!'

hello( 'a','b', 'c' )

'Hello, a b c!'

In [50]:
hello('a', 'b', 'c', sep='*')

'Hello, a*b*c!'

In [52]:
# It isn’t counted with the other arguments:

print( hello.__code__.co_argcount )

print( hello.__code__.co_kwonlyargcount ) # Listed here;

0
1


## Function checks at below places:<br>

• co_argcount — number of mandatory, positional arguments. <br>
• __defaults__ — values that make co_argcount flexible. <br>
• co_flags<br>
    - Do we assign extra positional args to *args? <br>
    - Do we assign extra keyword args to **kwargs? <br>
• co_kwonlyargcount — number of keyword-only args <br>

# How scoping works ?

- L -Local <br>
- E -Enclosing<br>
- G -Global<br>
- B -Builtins

In [54]:

x = 100

def func():
    print(f'In func, x = {x}')
    
print(f'Before, x = {x}')
func()
print(f'After, x = {x}')

Before, x = 100
In func, x = 100
After, x = 100


## How function knows, `x` is not local ? <br>

`__code__.co_varnames`

In [57]:
func.__code__.co_varnames # if x is not in co_varnames. Its not localvariables.

()

In [59]:

x = 100

def func():
    x = 200 # local
    print(f'In func, x = {x}')
    
print(f'Before, x = {x}')
func()
print(f'After, x = {x}')

Before, x = 100
In func, x = 200
After, x = 100


## How function knows, `x` is local ? <br>

*Note: co_varnames is populated at compile time, not runtime!

In [61]:
func.__code__.co_varnames

('x',)

In [65]:

x = 100

def func():
    print(f'In func, x = {x}' ) # python knows x is local. But, no Local value.
    x = 200
    
    
print(f'Before, x = {x}')
func()
print(f'After, x = {x}')


Before, x = 100


UnboundLocalError: local variable 'x' referenced before assignment

In [67]:

x = 100

def func():
    x += 1
    print(f'In func, x = {x}')
    
print(f'Before, x = {x}')
func()
print(f'After, x = {x}')

Before, x = 100


UnboundLocalError: local variable 'x' referenced before assignment

## Global declaration

In [69]:
x = 100
def func():
    global x
    x = 200 # converts to global declaration
    print(f'In func, x = {x}')
    
print(f'Before, x = {x}')
func()
print(f'After, x = {x}')

Before, x = 100
In func, x = 200
After, x = 200


## What does `global` do ?

It removes a variable from `co_varnames`.

In [70]:
func.__code__.co_varnames

()

Python uses LEGB to look for “x”. <br>
Once it cann't find the local variable names in the tuple. <br>
It assigns to the `global` variable.

In [73]:
dis.dis( func ) # Byte code with a global `x`; observe: LOAD_GLOBAL

  4           0 LOAD_CONST               1 (200)
              2 STORE_GLOBAL             0 (x)

  5           4 LOAD_GLOBAL              1 (print)
              6 LOAD_CONST               2 ('In func, x = ')
              8 LOAD_GLOBAL              0 (x)
             10 FORMAT_VALUE             0
             12 BUILD_STRING             2
             14 CALL_FUNCTION            1
             16 POP_TOP
             18 LOAD_CONST               0 (None)
             20 RETURN_VALUE


# `E` Enclosing<br>

function within function.

In [74]:

def outer():
    run_counter = 0
    total = 0
    
    def inner(x):
        run_counter += 1
        total += x
        print(f'Run {run_counter}, total is {total}')
        
    return inner


In [75]:
func = outer()

for i in range(10, 100, 10):
    func(i)

UnboundLocalError: local variable 'run_counter' referenced before assignment

In [76]:
# Make them non local;

def outer():
    run_counter = 0
    total = 0
    
    def inner(x):
        nonlocal run_counter, total
        run_counter += 1
        total += x
        print(f'Run {run_counter}, total is {total}')
        
    return inner


In [77]:
func = outer()
for i in range(10, 100, 10):
    func(i)

Run 1, total is 10
Run 2, total is 30
Run 3, total is 60
Run 4, total is 100
Run 5, total is 150
Run 6, total is 210
Run 7, total is 280
Run 8, total is 360
Run 9, total is 450


In [78]:
func = outer()
func.__code__.co_freevars


('run_counter', 'total')

In [79]:
# “outer” knows which of its local variables are referenced

outer.__code__.co_cellvars


('run_counter', 'total')

Summary:<br>

    def does two things:
        - Create a function object;
        - Assigns that function object to a variable.
    Function objects contain attributes;
    Scoping

    